In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
# Reference from above:
!mkdir temp

os.chdir("temp")
# In order for this to be work, sheet needs to be public
# Or "On, anyone with link can access"

# Metadata to Upload
!curl "https://docs.google.com/spreadsheets/d/1UkABgMlBIinJjITa6WepFAL-8VBkulS0LCbKojRXjVY/export?format=tsv" > current_metadata.tsv
current_mdata = pd.read_csv(os.getcwd()+"/current_metadata.tsv", sep="\t", index_col=[0])
current_mdata = current_mdata.transpose().set_index("Unique ID")

# Official Names
!curl "https://docs.google.com/spreadsheets/d/1ijQrjYv-mxtmoDHxv0S1XlfRed6IbYcviRW_HY7GAh8/export?format=tsv" > official_names.tsv
official_names = pd.read_csv(os.getcwd()+"/official_names.tsv", sep="\t", index_col=[0])

os.chdir("..")
!rm -r temp

In [ ]:
current_mdata

In [ ]:
official_names

In [ ]:
need_to_be_added = []

# Helper function
def list_to_string(lst):
    res = ""
    for ix, item in enumerate(lst):
        if ix < len(lst)-1:
            res += item + "/"
        else:
            res += item

    return(res)

def clean_sources(source_orgs):
    
    # Get list of just long-form names
    source_list = source_orgs.split("/")
    long_srcs = []
    
    for src in source_list:

        paren_finder = src.split("(")
        
        if (len(paren_finder) > 1):
            # accronym stored in second section
            long_name = paren_finder[0]
            long_name = long_name.strip()
            long_srcs.append(long_name)
        else:
            long_name = paren_finder[0].strip()
            long_srcs.append(long_name)
    
    # Add acronyms onto long-form names
    subtitles_list = []
    source_org_list = []
    
    for src in long_srcs:
        try:
#             if(src in changed_names):
#                 src = changed_names[src]
#                 print(src)
            accronym = official_names.loc[src]["Subtitle, Source (top of data page)"]
            #official_names.loc[src]["count"] += 1
        
        # This block runs if src is not in official names
        except:
            need_to_be_added.append(src)
            print("accronym not yet assigned")
        
            source_org_list.append(src + " (accronym not yet assigned)")
            subtitles_list.append("accronym not yet assigned")
                
        # If official acronym exists, this runs
        else:
            source_org_list.append(src + " (" + accronym + ")")
            subtitles_list.append(accronym)
                
    # strip brackets, replace with /
    clean_sources = list_to_string(source_org_list)
    clean_subtitles = list_to_string(subtitles_list)
    
    return(clean_sources, clean_subtitles)
    
def update_source_info(row):
    #print("Title: ", row["New Simple Title"])
    #print("Source Orgs: ", row["Source Organizations"])
    row["Source Organizations"], row["Subtitle"] = clean_sources(row["Source Organizations"])
    return(row)

In [ ]:
dummy = current_mdata.copy()
last = dummy.shape[0]
dummy = dummy.iloc[0:last-1]

In [ ]:
new_df = dummy.apply(lambda row: update_source_info(row), axis=1)

new_df

In [ ]:
np.unique(need_to_be_added)

In [ ]:
current_mdata